In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import os
# import torchvision.datasets as dset
# import torchvision.transforms as transforms
# import torchvision.utils as vutils
from torch.autograd import Variable
import torch.autograd as autograd
import numpy as np
import pescador
import librosa
import pprint
from IPython.display import Audio

In [2]:
class PhaseShuffle(nn.Module):
    """
    Performs phase shuffling, i.e. shifting feature axis of a 3D tensor
    by a random integer in {-n, n} and performing reflection padding where
    necessary
    """
    def __init__(self, n):
        super(PhaseShuffle, self).__init__()
        self.n = n
        
    def forward(self, x):
        # Make sure to use PyTorch to generate number RNG state is all shared
        k = int(torch.Tensor(1).random_(0,self.n + 1)) - 5
        
        # Return if no phase shift
        if k == 0:
            return x
        
        # Slice feature dimension
        if k > 0:
            x_trunc = x[:, :, :-k]
            pad = (0, k)
        else:
            x_trunc = x[:, :, -k:]
            pad = (-k, 0)
        
        # Reflection padding
        x_shuffle = F.pad(x_trunc, pad, mode='reflect')
        assert x_shuffle.shape == x.shape, "{}, {}".format(x_shuffle.shape, x.shape)
        return x_shuffle
        


class WaveGANGenerator(nn.Module):
    def __init__(self, d, ngpus, c=1, latent_dim=100, verbose=False):
        super(WaveGANGenerator, self).__init__()
        self.ngpus = ngpus
        self.d = d
        self.c = c
        self.latent_dim = latent_dim
        self.fc1 = nn.DataParallel(nn.Linear(latent_dim, 256*d))
        self.verbose = verbose
        
        self.tconv1 = nn.DataParallel(nn.ConvTranspose1d(16*d, 8*d, 25, stride=4, padding=11, output_padding=1))
        self.tconv2 = nn.DataParallel(nn.ConvTranspose1d(8*d, 4*d, 25, stride=4, padding=11, output_padding=1))
        self.tconv3 = nn.DataParallel(nn.ConvTranspose1d(4*d, 2*d, 25, stride=4, padding=11, output_padding=1))
        self.tconv4 = nn.DataParallel(nn.ConvTranspose1d(2*d, d, 25, stride=4, padding=11, output_padding=1))
        self.tconv5 = nn.DataParallel(nn.ConvTranspose1d(d, c, 25, stride=4, padding=11, output_padding=1))
        
        for m in self.modules():
            if isinstance(m, nn.ConvTranspose1d) or isinstance(m, nn.Linear):
                nn.init.kaiming_normal(m.weight.data)

    def forward(self, x):
        
        x = self.fc1(x).view(-1, 16*self.d, 16)
        x = F.relu(x)
        if self.verbose:
            print(x.shape)
            
        x = F.relu(self.tconv1(x))
        if self.verbose:
            print(x.shape)
            
        x = F.relu(self.tconv2(x))
        if self.verbose:
            print(x.shape)
            
        x = F.relu(self.tconv3(x))
        if self.verbose:
            print(x.shape)
            
        x = F.relu(self.tconv4(x))
        if self.verbose:
            print(x.shape)
            
        output = F.tanh(self.tconv5(x))
        if self.verbose:
            print(output.shape)

        return output

    
class WaveGANDiscriminator(nn.Module):
    def __init__(self, d, ngpus, c=1, n=2, alpha=0.2, verbose=False):
        super(WaveGANDiscriminator, self).__init__()
        self.d = d
        self.ngpus = ngpus
        self.c = c
        self.n = n
        self.alpha=alpha
        self.verbose = verbose
        # Conv2d(in_channels, out_channels, kernel_size, stride=1, etc.)
        self.conv1 = nn.DataParallel(nn.Conv1d(c, d, 25, stride=4, padding=11))
        self.conv2 = nn.DataParallel(nn.Conv1d(d, 2*d, 25, stride=4, padding=11))
        self.conv3 = nn.DataParallel(nn.Conv1d(2*d, 4*d, 25, stride=4, padding=11))
        self.conv4 = nn.DataParallel(nn.Conv1d(4*d, 8*d, 25, stride=4, padding=11))
        self.conv5 = nn.DataParallel(nn.Conv1d(8*d, 16*d, 25, stride=4, padding=11))
        self.ps1 = PhaseShuffle(n)
        self.ps2 = PhaseShuffle(n)
        self.ps3 = PhaseShuffle(n)
        self.ps4 = PhaseShuffle(n)
        self.fc1 = nn.DataParallel(nn.Linear(256*d, 1))
        
        for m in self.modules():
            if isinstance(m, nn.Conv1d) or isinstance(m, nn.Linear):
                nn.init.kaiming_normal(m.weight.data)

    def forward(self, x):
        x = F.leaky_relu(self.conv1(x), negative_slope=self.alpha)
        if self.verbose:
            print(x.shape)
        x = self.ps1(x)
        
        x = F.leaky_relu(self.conv2(x), negative_slope=self.alpha)
        if self.verbose:
            print(x.shape)            
        x = self.ps2(x)
        
        x = F.leaky_relu(self.conv3(x), negative_slope=self.alpha)
        if self.verbose:
            print(x.shape)
        x = self.ps3(x)
        
        x = F.leaky_relu(self.conv4(x), negative_slope=self.alpha)
        if self.verbose:
            print(x.shape)            
        x = self.ps4(x)
        
        
        x = F.leaky_relu(self.conv5(x), negative_slope=self.alpha)
        if self.verbose:
            print(x.shape)
            
        x = x.view(-1, 256*self.d)
        if self.verbose:
            print(x.shape)
            
        return F.sigmoid(self.fc1(x))

In [3]:
# Create generator
latent_dim = 100
gen = WaveGANGenerator(d=64, ngpus=0, c=1, latent_dim=latent_dim).cuda()

In [4]:
# Create discriminator
disc = WaveGANDiscriminator(d=64, ngpus=0).cuda()

In [5]:
# Sample from noise distribution p(z)
z = torch.Tensor(5, latent_dim).uniform_(0, 1)
z = Variable(z)
z = z.cuda()

In [6]:
# Run the sample through the generator to generate a sample
# from the model distribution
out = gen(z)

In [7]:
# Evaluate the given waveforms with the discriminator
disc(out)

Variable containing:
 0.6283
 0.6747
 0.7275
 0.5735
 0.7001
[torch.cuda.FloatTensor of size 5x1 (GPU 0)]

In [8]:
def file_sample_generator(filepath, window_length=16384, fs=16000):
    """
    Audio sample generator
    """
    try:
        audio_data, _ = librosa.load(filepath, sr=fs)
    except Exception as e:
        raise StopIteration()
        
    audio_len = len(audio_data)
    
    # Pad audio to at least a single frame
    if audio_len < window_length:
        pad_length = window_length - audio_len
        left_pad = pad_length // 2
        right_pad = pad_length - left_pad
        
        audio_data = np.pad(audio_data, (left_pad, right_pad), mode='constant')
        audio_len = len(audio_data)
        
    while True:
        if audio_len == window_length:
            # If we only have a single frame's worth of audio, just yield the whole audio
            sample = audio_data
        else:
            # Sample a random window from the audio file
            start_idx = np.random.randint(0,audio_len - window_length)
            end_idx = start_idx + window_length
            sample = audio_data[start_idx:end_idx]
            
        sample = sample.astype('float32')
            
        try:
            assert not np.any(np.isnan(sample))
            
        except Type:e
            
        yield {'X': sample}
    
def create_batch_generator(audio_filepath_list, batch_size):
    streamers = []
    for audio_filepath in audio_filepath_list:
        s = pescador.Streamer(file_sample_generator, audio_filepath)
        streamers.append(s)
        
    mux = pescador.ShuffledMux(streamers)
    batch_gen = pescador.buffer_stream(mux, batch_size)
    
    return batch_gen

In [9]:
def get_all_audio_filepaths(audio_dir):
    return [os.path.join(root, fname)
            for (root, dir_names, file_names) in os.walk(audio_dir)
            for fname in file_names
            if fname.lower().endswith('.wav')]

In [10]:
def create_data_split(audio_filepath_list, valid_ratio, test_ratio, train_batch_size, valid_size, test_size):
    num_files = len(audio_filepath_list)
    num_valid = int(np.ceil(num_files * valid_ratio))
    num_test = int(np.ceil(num_files * test_ratio))
    num_train = num_files - num_valid - num_test
    
    assert num_valid > 0
    assert num_test > 0
    assert num_train > 0
    
    valid_files = audio_filepath_list[:num_valid]
    test_files = audio_filepath_list[num_valid:num_valid+num_test]
    train_files = audio_filepath_list[num_valid+num_test:]
    train_files, valid_files, test_files = create_data_split(audio_filepath_list, valid_ratio, test_ratio)
    
    train_gen = create_batch_generator(train_files, train_batch_size)
    valid_data = next(iter(create_batch_generator(valid_files, valid_size)))
    test_data = next(iter(create_batch_generator(train_files, test_size)))
    
    return train_gen, valid_data, test_data

In [11]:
# Adapted from https://github.com/caogang/wgan-gp/blob/master/gan_toy.py
def calc_gradient_penalty(model_dis, real_data, fake_data, batch_size, lmbda, use_cuda=True):
    # Compute interpolation factors
    alpha = torch.rand(batch_size, 1, 1)
    alpha = alpha.expand(real_data.size())
    alpha = alpha.cuda() if use_cuda else alpha

    # Interpolate between real and fake data
    interpolates = alpha * real_data + ((1 - alpha) * fake_data)
    if use_cuda:
        interpolates = interpolates.cuda()
    interpolates = autograd.Variable(interpolates, requires_grad=True)

    # Evaluate discriminator
    disc_interpolates = model_dis(interpolates)

    # Obtain gradients of the discriminator with respect to the inputs
    gradients = autograd.grad(outputs=disc_interpolates, inputs=interpolates,
                              grad_outputs=torch.ones(disc_interpolates.size()).cuda() if use_cuda else torch.ones(
                                  disc_interpolates.size()),
                              create_graph=True, retain_graph=True, only_inputs=True)[0]

    # Compute MSE between 1.0 and the gradient of the norm penalty to encourage discriminator
    # to be a 1-Lipschitz function
    gradient_penalty = ((gradients.norm(2, dim=1) - 1) ** 2).mean() * lmbda
    return gradient_penalty

In [12]:
# Adapted from https://github.com/caogang/wgan-gp/blob/master/gan_toy.py
def train_wavegan(train_gen, valid_data, test_data, num_epochs, batches_per_epoch,
                  batch_size, lmbda=0.1, ngpus=1, model_size=64, discriminator_updates=5, epochs_per_sample=10,
                  sample_size=20, loss='wgan-gp', lr=1e-4, beta_1=0.5, beta_2=0.9, latent_dim=100):
    # TODO: Incorporate validation and test data

    use_cuda = ngpus >= 1
    
    # Build the models
    model_gen = WaveGANGenerator(model_size, ngpus, c=1, latent_dim=latent_dim)
    model_dis = WaveGANDiscriminator(model_size, ngpus, c=1, n=2, alpha=0.2)

    if use_cuda:
        model_gen = model_gen.cuda()
        model_dis = model_dis.cuda()
    
    # Convenient values for 
    one = torch.FloatTensor([1])
    neg_one = one * -1
    if use_cuda:
        one = one.cuda()
        neg_one = neg_one.cuda()
    
    # Initialize optimizers for each model
    optimizer_gen = optim.Adam(model_gen.parameters(), lr=lr, betas=(beta_1, beta_2))
    optimizer_dis = optim.Adam(model_dis.parameters(), lr=lr, betas=(beta_1, beta_2))

    # Sample noise used for seeing the evolution of generated output samples throughout training
    sample_noise = torch.randn(sample_size, latent_dim)
    if use_cuda:
        sample_noise = sample_noise.cuda()
    sample_noisev = autograd.Variable(sample_noise)
    
    samples = {}
    history = []
    
    train_iter = iter(train_gen)
    
    # Loop over the dataset multiple times
    for epoch in range(num_epochs):
        print("Epoch: {}".format(epoch+1))

        
        
        epoch_history = []

        for batch_idx in range(batches_per_epoch):

            # Set model parameters to require gradients to be computed and stored
            for p in model_dis.parameters():
                p.requires_grad = True

            # Initialize the metrics for this batch
            batch_history = {
                'discriminator': [],
                'generator': {}
            }
            
            # Discriminator Training Phase:
            # -> Train discriminator k times
            for iter_d in range(discriminator_updates):
                # Get new batch of real training data
                real_data = next(train_iter)['X']
                real_data = real_data[:,np.newaxis,:]
                real_data = torch.Tensor(real_data)
                if use_cuda:
                    real_data = real_data.cuda()
                real_data_v = autograd.Variable(real_data)

                # Reset gradients
                model_dis.zero_grad()

                # a) Compute loss contribution from real training data and backprop
                # (negative of the empirical mean, w.r.t. the data distribution, of the discr. output)
                D_real = model_dis(real_data_v)
                D_real = D_real.mean()
                
                # Negate since we want to _maximize_ this quantity
                D_real.backward(neg_one)
                

                # b) Compute loss contribution from generated data and backprop
                # (empirical mean, w.r.t. the generator distribution, of the discr. output)
                # Generate noise in latent space
                noise = torch.randn(batch_size, latent_dim)
                if use_cuda:
                    noise = noise.cuda()
                noisev = autograd.Variable(noise, volatile=True)  # totally freeze model_gen
                # Generate data by passing noise through the generator
                fake = autograd.Variable(model_gen(noisev).data)
                inputv = fake
                D_fake = model_dis(inputv)
                D_fake = D_fake.mean()
                D_fake.backward(one)

                # c) Compute gradient penalty and backprop
                gradient_penalty = calc_gradient_penalty(model_dis, real_data_v.data, fake.data,
                                                         batch_size, lmbda, use_cuda=use_cuda)
                gradient_penalty.backward()
                
                # d) Update the discriminator
                optimizer_dis.step()
                
                # Compute metrics and record in batch history
                D_cost = D_fake - D_real + gradient_penalty
                Wasserstein_D = D_real - D_fake

                if use_cuda:
                    D_cost = D_cost.cpu()
                    Wasserstein_D = Wasserstein_D.cpu()
                
                batch_history['discriminator'].append({
                    'cost': D_cost.data.numpy()[0],
                    'wasserstein_cost': Wasserstein_D.data.numpy()[0]
                })

            ############################
            # (2) Update G network
            ###########################
            
            # Prevent discriminator from computing gradients, since
            # we are only updating the generator
            for p in model_dis.parameters():
                p.requires_grad = False

            # Reset generator gradients
            model_gen.zero_grad()

            # Sample from the generator
            noise = torch.randn(batch_size, latent_dim)
            if use_cuda:
                noise = noise.cuda()
            noisev = autograd.Variable(noise)
            fake = model_gen(noisev)
            
            # Compute generator loss and backprop
            # (negative of empirical mean (w.r.t generator distribution) of discriminator output)
            G = model_dis(fake)
            G = G.mean()
            G.backward(neg_one)
            G_cost = -G

            # Update generator
            optimizer_gen.step()
            
            if use_cuda:
                G_cost = G_cost.cpu()

            # Record generator loss
            batch_history['generator']['cost'] = G_cost.data.numpy()[0]
            
            # Record batch metrics
            epoch_history.append(batch_history)
            
        # Record epoch metrics
        history.append(epoch_history)
        
        pprint.pprint(epoch_history)
        print("\n")
        
        if (epoch + 1) % epochs_per_sample == 0:
            # Generate outputs for fixed latent samples
            samp_output = model_gen(sample_noisev)
            if use_cuda:
                samp_output = samp_output.cpu()
            samples[epoch] = samp_output.data.numpy()
            
    return model_gen, model_dis, history, samples

In [ ]:
# Try on some training data
batch_size = 64
# gen1 = create_batch_generator(["/scratch/nl1115/Bach_Lute.mp3"], batch_size)
# valid_data = None
# test_data = None
audio_dir = "/beegfs/jtc440/aml/TheDrumClub-Kit004-THEMEGABUNDLE"
audio_filepaths = get_all_audio_filepaths(audio_dir)
# gen1, valid_data, test_data = create_data_split(audio_filepaths, valid_ratio, test_ratio,
#                                                 batch_size, 64, 512)
gen1 = create_batch_generator(audio_filepaths, batch_size)
valid_data = None
test_data = None

model_gen, model_dis, history, samples = train_wavegan(train_gen=gen1, 
                                                       valid_data=valid_data, 
                                                       test_data=test_data, 
                                                       num_epochs=5, 
                                                       batches_per_epoch=8,
                                                       batch_size=batch_size, 
                                                       lmbda=10, 
                                                       ngpus=4, 
                                                       model_size=64,
                                                       discriminator_updates=1,
                                                       latent_dim=100,
                                                       epochs_per_sample=1,
                                                       sample_size=20, 
                                                       loss='wgan-gp') 

Epoch: 1
[{'discriminator': [{'cost': 10.159609, 'wasserstein_cost': -0.19903404}],
  'generator': {'cost': -1.2866402e-06}},
 {'discriminator': [{'cost': 9.577552, 'wasserstein_cost': 0.41412827}],
  'generator': {'cost': -7.2360795e-05}},
 {'discriminator': [{'cost': 9.437906, 'wasserstein_cost': 0.5263806}],
  'generator': {'cost': -5.242857e-06}},
 {'discriminator': [{'cost': 9.293784, 'wasserstein_cost': 0.69623244}],
  'generator': {'cost': -3.9252836e-07}},
 {'discriminator': [{'cost': 9.099351, 'wasserstein_cost': 0.8898482}],
  'generator': {'cost': -4.4458062e-07}},
 {'discriminator': [{'cost': 9.019555, 'wasserstein_cost': 0.9722519}],
  'generator': {'cost': -1.7118346e-06}},
 {'discriminator': [{'cost': 8.9902725, 'wasserstein_cost': 0.99652445}],
  'generator': {'cost': -0.00021928802}},
 {'discriminator': [{'cost': 10.00125, 'wasserstein_cost': -0.0012849569}],
  'generator': {'cost': -0.99999994}}]


Epoch: 2
[{'discriminator': [{'cost': 10.000479, 'wasserstein_cost': -

In [15]:
Audio(filename=audio_filepaths[0])

In [22]:
Audio(data=samples[4][18,0], rate=16000)